In [1507]:
from amplpy import AMPL
import pandas as pd

In [1508]:
rideshares = pd.read_csv('./rideshare_tg22.csv')
airports = ['LAX', 'ONT', 'SNA', 'BUR']
car_types = ['UberX', 'UberXL']

In [1509]:
ez_rideshares = rideshares#[rideshares['Departure Date'] == '5/11/24']
ez_rideshares.rename(columns={'Departure Time (Pacific Daylight Time)': 'Flight depart'}, inplace=True)

In [1510]:
#convert hour:minutes to number of hours 
ez_rideshares['Earliest Airport Arrival Time'] = ez_rideshares['Earliest Airport Arrival Time'].str.split(':').apply(lambda x: int(x[0]) + int(x[1])/60)
ez_rideshares['Latest Airport Arrival Time'] = ez_rideshares['Latest Airport Arrival Time'].str.split(':').apply(lambda x: int(x[0]) + int(x[1])/60)

# Rounded to 2 decimal places
ez_rideshares['Earliest Airport Arrival Time'] = ez_rideshares['Earliest Airport Arrival Time'].round(2)
ez_rideshares['Latest Airport Arrival Time'] = ez_rideshares['Latest Airport Arrival Time'].round(2)

# create df for student earliest arrival time and student latest arrival time to the airport with student name as index

earliest_arrival_time = pd.DataFrame({ 'Name': ez_rideshares['Name'], 'Earliest Airport Arrival Time': ez_rideshares['Earliest Airport Arrival Time'] })
earliest_arrival_time.set_index('Name', inplace=True)

latest_arrival_time=pd.DataFrame({ 'Name': ez_rideshares['Name'], 'Latest Airport Arrival Time': ez_rideshares['Latest Airport Arrival Time'] })
latest_arrival_time.set_index('Name', inplace=True)



In [1511]:
latest_arrival_time

,Latest Airport Arrival Time
Name,
Nithya Yeluri,5.50
Nina Jobanputra,11.25
Aanya Pratapneni,11.50
Will Sedo,12.50
Marcella Todd,13.00
Kaanthi Pandhigunta,13.75
Cevi Bainton,15.00
Ally Dye,15.00
Maya Maranto,17.25


In [1512]:
cost_data = {
    ('LAX', 'UberX'): 90, ('LAX', 'UberXL'): 120,
    ('ONT', 'UberX'): 30, ('ONT', 'UberXL'): 40,
    ('SNA', 'UberX'): 90, ('SNA', 'UberXL'): 120,
    ('BUR', 'UberX'): 90, ('BUR', 'UberXL'): 120
}

In [1513]:
capacity_data = {
    'UberX': 4,
    'UberXL': 6
}

In [1514]:
travel_time_data = {
    'LAX': 1.5,
    'ONT': 0.5,
    'SNA': 1.5,
    'BUR': 1.5
}

In [1515]:
pricing_multiplier = [1.00, 1.02, 1.02, 1.03, 1.03, 0.99, 1.04, 1.03, 1.03, 1.08, 1.08, 1.14, 1.14, 1.19, 1.19, 1.20, 1.20, 1.30, 1.14, 1.10, 1.13, 1.12, 1.15, 1.17, 1.16, 1.17, 1.17, 1.20, 1.20, 1.19, 1.23, 1.22, 1.26, 1.24, 1.27, 1.25, 1.25, 1.18, 1.18, 1.19, 1.19, 1.17, 1.08, 1.12, 1.12, 1.07, 1.07, 1.00 ]

In [1516]:
ez_rideshares['Flight depart'] = pd.to_datetime(ez_rideshares['Flight depart'], format='%H:%M')
ez_rideshares['Flight times'] = ez_rideshares['Flight depart'].dt.hour + 0.5 * (ez_rideshares['Flight depart'].dt.minute//30)

In [1517]:
ampl = AMPL()
ampl.eval(r"""
    set A; #airports
    set S; #students
    set P = 0.5..24 by 0.5;  # time leaving to airport
    set C; #car types

    param cost{A, C}; # cost of car type c to airport a
    param capacity{C}; # capacity of car type c
    param travelTime{A};    # time to travel to airport a
    param flightTime{S, A}; # time of flight for student s at airport a
    param earliestArrivalTime{S}; # earliest arrival time for student s
    param latestArrivalTime{S}; # latest arrival time for student s

    var numCars{A, P, C} >= 0 integer;  # x
    var leaving{A, S, P} binary;  # 1 if student s leaves for airport a at time p, else 0
    var departureTime{A, S}; # time student s leaves for airport a

    minimize totalCost: 
        sum{a in A, p in P, c in C} numCars[a, p, c]*cost[a, c];
    

    subject to notTooLateConstraint{a in A, s in S, p in P}:
        p - leaving[a, s, p]*(latestArrivalTime[s] - travelTime[a]) <= 100*(1 - leaving[a, s, p]);

    subject to notTooEarlyConstraint{a in A, s in S, p in P}:
        leaving[a, s, p]*(earliestArrivalTime[s] - travelTime[a]) - p <= 100*(1 - leaving[a, s, p]);

    subject to capacityConstraint{a in A, p in P}:
        sum{s in S} leaving[a, s, p] <= sum{c in C} numCars[a, p, c] * capacity[c];

    subject to onlyLeaveOnce{s in S}:
        sum{a in A, p in P} leaving[a, s, p] = 1;

    subject to defineDepartureTime{a in A, s in S}:
        departureTime[a, s] = sum{p in P} leaving[a, s, p] * p;

    subject to CorrectAirport{a in A, s in S, p in P}: #bad name but it works
        leaving[a,s,p] * (departureTime[a, s] + travelTime[a]) <= leaving[a,s,p] * (flightTime[s, a]);
        
          
""")

In [1518]:
# reset index on ez_rideshares to start at 0
ez_rideshares.reset_index(drop=True, inplace=True)

ez_rideshares

,Name,Airport,Departure Date,Flight depart,Earliest Airport Arrival Time,Latest Airport Arrival Time,Flight times
0,Nithya Yeluri,ONT,11/22/2022,1900-01-01 07:00:00,5.5,5.50,7.0
1,Nina Jobanputra,ONT,11/22/2022,1900-01-01 12:15:00,11.0,11.25,12.0
2,Aanya Pratapneni,ONT,11/22/2022,1900-01-01 13:25:00,11.0,11.50,13.0
3,Will Sedo,ONT,11/22/2022,1900-01-01 13:29:00,11.5,12.50,13.0
4,Marcella Todd,ONT,11/22/2022,1900-01-01 15:00:00,11.0,13.00,15.0
5,Kaanthi Pandhigunta,ONT,11/22/2022,1900-01-01 14:45:00,12.0,13.75,14.5
6,Cevi Bainton,ONT,11/22/2022,1900-01-01 15:50:00,13.0,15.00,15.5
7,Ally Dye,ONT,11/22/2022,1900-01-01 16:15:00,14.0,15.00,16.0
8,Maya Maranto,ONT,11/22/2022,1900-01-01 18:05:00,16.5,17.25,18.0
9,Alec Vercruysse,ONT,11/22/2022,1900-01-01 17:55:00,16.5,17.17,17.5


In [1519]:
# set flight time data equal to Flight Times form ez_rideshares
# format: {(ez_rideshares['Name'][i], 'ONT') : ez_rideshares['Flight times'][i]}

raw_flight_time_data = {}
for i in range(len(ez_rideshares)):
    raw_flight_time_data[(ez_rideshares['Name'][i], ez_rideshares['Airport'][i])] = ez_rideshares['Flight times'][i]

raw_flight_time_data

{('Nithya Yeluri', 'ONT'): 7.0,
 ('Nina Jobanputra', 'ONT'): 12.0,
 ('Aanya Pratapneni ', 'ONT'): 13.0,
 ('Will Sedo', 'ONT'): 13.0,
 ('Marcella Todd', 'ONT'): 15.0,
 ('Kaanthi Pandhigunta', 'ONT'): 14.5,
 ('Cevi Bainton', 'ONT'): 15.5,
 ('Ally Dye', 'ONT'): 16.0,
 ('Maya Maranto', 'ONT'): 18.0,
 ('Alec Vercruysse', 'ONT'): 17.5,
 ('Rebecca Preis', 'ONT'): 18.5,
 ('Susan Li', 'ONT'): 19.5,
 ('Tanvi Krishnan ', 'ONT'): 20.5,
 ('Jennifer Li', 'ONT'): 20.5,
 ('Allison Marten', 'ONT'): 20.5,
 ('Aldrin Feliciano', 'ONT'): 21.0,
 ('Julianna Schalkwyk', 'ONT'): 23.5,
 ('Charles Weismann', 'ONT'): 5.0,
 ('Katrina Nguyen', 'ONT'): 6.0,
 ('Bennet Matazzoni', 'ONT'): 15.5,
 ('Thaxter Shaw', 'LAX'): 22.5,
 ('Alicia Krasner', 'LAX'): 13.0,
 ('Laura Vairus', 'LAX'): 17.5,
 ('Ashley Kim', 'SNA'): 21.0}

In [1520]:
flight_time_data = {}
for s in ez_rideshares['Name']:
    for a in airports:
        if (s, a) in raw_flight_time_data:
            flight_time_data[(s, a)] = raw_flight_time_data[(s, a)]
        else:
            flight_time_data[(s, a)] = 0

flight_time_data

{('Nithya Yeluri', 'LAX'): 0,
 ('Nithya Yeluri', 'ONT'): 7.0,
 ('Nithya Yeluri', 'SNA'): 0,
 ('Nithya Yeluri', 'BUR'): 0,
 ('Nina Jobanputra', 'LAX'): 0,
 ('Nina Jobanputra', 'ONT'): 12.0,
 ('Nina Jobanputra', 'SNA'): 0,
 ('Nina Jobanputra', 'BUR'): 0,
 ('Aanya Pratapneni ', 'LAX'): 0,
 ('Aanya Pratapneni ', 'ONT'): 13.0,
 ('Aanya Pratapneni ', 'SNA'): 0,
 ('Aanya Pratapneni ', 'BUR'): 0,
 ('Will Sedo', 'LAX'): 0,
 ('Will Sedo', 'ONT'): 13.0,
 ('Will Sedo', 'SNA'): 0,
 ('Will Sedo', 'BUR'): 0,
 ('Marcella Todd', 'LAX'): 0,
 ('Marcella Todd', 'ONT'): 15.0,
 ('Marcella Todd', 'SNA'): 0,
 ('Marcella Todd', 'BUR'): 0,
 ('Kaanthi Pandhigunta', 'LAX'): 0,
 ('Kaanthi Pandhigunta', 'ONT'): 14.5,
 ('Kaanthi Pandhigunta', 'SNA'): 0,
 ('Kaanthi Pandhigunta', 'BUR'): 0,
 ('Cevi Bainton', 'LAX'): 0,
 ('Cevi Bainton', 'ONT'): 15.5,
 ('Cevi Bainton', 'SNA'): 0,
 ('Cevi Bainton', 'BUR'): 0,
 ('Ally Dye', 'LAX'): 0,
 ('Ally Dye', 'ONT'): 16.0,
 ('Ally Dye', 'SNA'): 0,
 ('Ally Dye', 'BUR'): 0,
 ('Maya M

In [1521]:
# read ez_rideshares into ampl2
ampl.set['A'] = airports
ampl.set['S'] = ez_rideshares['Name']
ampl.set['C'] = car_types

ampl.getParameter("cost").setValues(cost_data)
ampl.getParameter("capacity").setValues(capacity_data)
ampl.getParameter("travelTime").setValues(travel_time_data)
ampl.getParameter("flightTime").setValues(flight_time_data)
ampl.getParameter("earliestArrivalTime").setValues(earliest_arrival_time)
ampl.getParameter("latestArrivalTime").setValues(latest_arrival_time)


In [1522]:
# run the model
ampl.solve(solver='gurobi')



Gurobi 11.0.3:Gurobi 11.0.3: optimal solution; objective 630
15 simplex iterations
1 branching node


In [1523]:
# display the results
student_airport_times = {}
for student in ez_rideshares['Name']:
    for a in airports:
        x = ampl.getValue(f'departureTime["{a}", "{student}"]')
        if x != 0:
            student_airport_times[student] = (a, x)
student_airport_times

{'Nithya Yeluri': ('ONT', 5),
 'Nina Jobanputra': ('ONT', 10.5),
 'Aanya Pratapneni ': ('ONT', 10.5),
 'Will Sedo': ('ONT', 11.5),
 'Marcella Todd': ('ONT', 10.5),
 'Kaanthi Pandhigunta': ('ONT', 11.5),
 'Cevi Bainton': ('ONT', 14),
 'Ally Dye': ('ONT', 14),
 'Maya Maranto': ('ONT', 16.5),
 'Alec Vercruysse': ('ONT', 16.5),
 'Rebecca Preis': ('ONT', 16.5),
 'Susan Li': ('ONT', 16.5),
 'Tanvi Krishnan ': ('ONT', 18.5),
 'Jennifer Li': ('ONT', 18.5),
 'Allison Marten': ('ONT', 18.5),
 'Aldrin Feliciano': ('ONT', 18.5),
 'Julianna Schalkwyk': ('ONT', 21),
 'Charles Weismann': ('ONT', 2),
 'Katrina Nguyen': ('ONT', 3.5),
 'Bennet Matazzoni': ('ONT', 14),
 'Thaxter Shaw': ('LAX', 16.5),
 'Alicia Krasner': ('LAX', 9.5),
 'Laura Vairus': ('LAX', 12.5),
 'Ashley Kim': ('SNA', 17)}

In [1524]:
for i in range(1, 49):
    for a in airports:
        for car in car_types:
            numCars = ampl.getValue(f"numCars['{a}', {i/2}, '{car}']")
            if numCars > 0:
                print(f'Time {i/2} has {numCars} {car} car(s) to {a}')

Time 2.0 has 1 UberX car(s) to ONT
Time 3.5 has 1 UberX car(s) to ONT
Time 5.0 has 1 UberX car(s) to ONT
Time 9.5 has 1 UberX car(s) to LAX
Time 10.5 has 1 UberX car(s) to ONT
Time 11.5 has 1 UberX car(s) to ONT
Time 12.5 has 1 UberX car(s) to LAX
Time 14.0 has 1 UberX car(s) to ONT
Time 16.5 has 1 UberX car(s) to LAX
Time 16.5 has 1 UberX car(s) to ONT
Time 17.0 has 1 UberX car(s) to SNA
Time 18.5 has 1 UberX car(s) to ONT
Time 21.0 has 1 UberX car(s) to ONT
